In [118]:
import requests
res = requests.get('http://localhost:8055')
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



### ES Auth

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost'], port=8055)
print es

<Elasticsearch([{u'host': 'localhost', u'scheme': 'http'}])>


In [2]:
result = es.search(index='agg', doc_type='count', size=1000000, body={
    'query': {
        "match_all" : {}
    }
})

lst = [x for x in result['hits']['hits']]
print len(lst)
for l in lst[:10]:
    print l

158986
{u'_score': 1.0, u'_type': u'count', u'_id': u'2812', u'_source': {u'category': u'baby_back_ribs', u'date': 20170319, u'datetime_date': u'2017-03-19', u'count': 1, u'country': u'saudi arabia'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'2813', u'_source': {u'category': u'baby_back_ribs', u'date': 20170404, u'datetime_date': u'2017-04-04', u'count': 1, u'country': u'saudi arabia'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'2819', u'_source': {u'category': u'baby_back_ribs', u'date': 20170611, u'datetime_date': u'2017-06-11', u'count': 1, u'country': u'saudi arabia'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'2820', u'_source': {u'category': u'baby_back_ribs', u'date': 20170626, u'datetime_date': u'2017-06-26', u'count': 1, u'country': u'saudi arabia'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'2821', u'_source': {u'category': u'baby_back_ribs', u'date': 20170629, u'datetime_date': u'201

## Query by

In [125]:
from itertools import groupby
from collections import defaultdict
from datetime import datetime, timedelta
from collections import Counter
from scipy import stats
import numpy as np
# the interval is currently implicit
dateBegin = 0
dateEnd = 20180400
category = 'sushi'
# country = 'united states'
country = 'italy'

In [126]:
# Query by food category - Fig. 2b,c Map/Report of Trends/Popularity
def query_by_category(category):
    result = es.search(index='agg', doc_type='count', size=100000, body={
        "query": {        
            "bool" : {                
                "filter": [
                    {"term" : { "category.keyword" : category }},
                    {"range": {            
                        "date" : {            
                            "gte" : dateBegin,
                            "lte" : dateEnd
                        }            
                    }}
                ]            
            }
        }
    })
    return result

In [127]:
# Query by food category - Fig. 2a Report of Trends/Popularity
def query_by_country(country):
    result = es.search(index='agg', doc_type='count', size=100000, body={
        "query": {        
            "bool" : {                
                "filter": [
                    {"term" : { "country.keyword" : country }},
                    {"range": {            
                        "date" : {            
                            "gte" : dateBegin,
                            "lte" : dateEnd
                        }            
                    }}
                ]            
            }
        }
    })
    return result

## Trends / Popularity based on query_by_category

In [ ]:
# get trends 
result = query_by_category(category)


def get_total_freq_country(country):
    result_country = query_by_country(country)
    total = 0
    for x in result_country['hits']['hits']:
        print x
        total += x["_source"]['count']
    return total

def get_trend_per_country(query_result, analysis_type="trend", interval=None):
    country_dict = defaultdict(list)
    for x in query_result['hits']['hits']:
        country_dict[x["_source"]['country']].append(tuple((x["_source"]['date'],x["_source"]['count'])))
    
    country_trend = defaultdict(float)
    for country, values in country_dict.iteritems():
        # sort list by date ascending
        sorted_counts = sorted(values, key=lambda x: x[0])
        # we refer to the current day in the interval
        if analysis_type == "frequency":
            country_trend[country] = sum([y for x,y in sorted_counts])
        elif analysis_type == "relative_frequency":
            total_sum = get_total_freq_country(country)
            country_trend[country] = sum([y for x,y in sorted_counts])/(1.0*total_sum)
            print country, sum([y for x,y in sorted_counts]), total_sum
        elif analysis_type == "popularity":
            if interval is None:
                y = [y for x, y in sorted_counts]
            else:
                new_intervals = split_intervat_in_buckets(interval, sorted_counts)
                print new_intervals
                y = [y for x, y in new_intervals]
            country_trend[country] = stats.zscore(y)[-1] 
        elif analysis_type == "trend":
            if interval is None:
                y = [y for x, y in sorted_counts]
            else:
                new_intervals = split_intervat_in_buckets(interval, sorted_counts)
                print new_intervals
                y = [y for x, y in new_intervals]
            x = np.arange(len(y))
            try:
                regression = np.polyfit(x, y, 1)
                country_trend[country] = regression[0]
            except:
                # the fir doesn't work for this example: np.polyfit(np.array([0]), np.array([1]), 1)
                # some versions of numpy return nan, as should we
                country_trend[country] = np.nan
            
    return country_trend

newd = get_trend_per_country(result, analysis_type="relative_frequency")
# newd = get_category_trend_per_country(result, analysis_type="popularity")
print [(x,y) for x,y in newd.iteritems()]

# WHY zscore is NaN - division by 0
# sorted_counts_lit = sorted(country_dict['lithuania'], key=lambda x: x[0])
# print stats.zscore([y for x,y in sorted_counts_lit])
# print sorted_counts_lit

## Trends / Popularity based on query_by_country

In [132]:
# get trends
result = query_by_country(country)

# this is dynamic but we can computer static totals and pass them as params
def get_total_freq_category(category):
    result_categ = query_by_category(category)
    total = 0
    for x in result_categ['hits']['hits']:
        total += x["_source"]['count']
    return total

def get_trend_per_category(query_result, analysis_type="trend", interval=None):
    category_dict = defaultdict(list)
    for x in query_result['hits']['hits']:
        category_dict[x["_source"]['category']].append(tuple((x["_source"]['date'], x["_source"]['count'])))

    category_trend = defaultdict(float)
    for category, values in category_dict.iteritems():
        # sort list by date ascending
        sorted_counts = sorted(values, key=lambda x: x[0])
        # we refer to the current day in the interval
        if analysis_type == "frequency":
            category_trend[category] = sum([y for x, y in sorted_counts])
        elif analysis_type == "relative_frequency":
            total_sum = get_total_freq_category(category)
            category_trend[category] = sum([y for x,y in sorted_counts])/(1.0*total_sum)
            print category, sum([y for x,y in sorted_counts]), total_sum
        elif analysis_type == "popularity":
            if interval is None:
                y = [y for x, y in sorted_counts]
            else:
                new_intervals = split_intervat_in_buckets(interval, sorted_counts)
                print new_intervals
                y = [y for x, y in new_intervals]
            category_trend[category] = stats.zscore(y)[-1]
        elif analysis_type == "trend":
            if interval is None:
                y = [y for x, y in sorted_counts]
            else:
                new_intervals = split_intervat_in_buckets(interval, sorted_counts)
                print new_intervals
                y = [y for x, y in new_intervals]
            x = np.arange(len(y))
            try:
                regression = np.polyfit(x, y, 1)
                category_trend[category] = regression[0]
            except:
                # the for doesn't work for this example: np.polyfit(np.array([0]), np.array([1]), 1)
                # some versions of numpy return nan, as should we
                category_trend[category] = np.nan

    return category_trend

newd = get_trend_per_category(result, analysis_type="popularity", interval=30)
# newd = get_trend_per_category(result, analysis_type="popularity")
print [(x,y) for x,y in newd.iteritems()]

# WHY zscore is NaN - division by 0
# sorted_counts_lit = sorted(country_dict['lithuania'], key=lambda x: x[0])
# print stats.zscore([y for x,y in sorted_counts_lit])
# print sorted_counts_lit

[(0, 17), (1, 18), (2, 6), (3, 9)]
[(0, 3), (2, 1), (3, 1)]
[(0, 48), (1, 46), (2, 15), (3, 56)]
[(0, 6), (2, 1)]
[(0, 22), (1, 43), (2, 17), (3, 19)]
[(0, 2), (1, 6), (2, 1), (3, 3)]
[(0, 2), (1, 3), (3, 5)]
[(0, 16), (1, 31), (2, 11), (3, 30)]
[(0, 75), (1, 85), (2, 42), (3, 72)]
[(0, 4), (2, 3), (3, 1)]
[(0, 58), (1, 53), (2, 12), (3, 54)]
[(0, 58), (1, 129), (2, 56), (3, 77)]
[(0, 2), (1, 5), (3, 2)]
[(0, 2), (1, 1), (2, 1)]
[(0, 4), (1, 15), (2, 26), (3, 5)]
[(0, 3)]
[(0, 557), (1, 827), (2, 379), (3, 450)]
[(0, 52), (1, 93), (2, 28), (3, 73)]
[(0, 83), (1, 104), (2, 33), (3, 63)]
[(0, 14), (1, 14), (2, 4), (3, 17)]
[(0, 1), (2, 1), (3, 1)]
[(0, 3), (1, 4), (2, 2), (3, 9)]
[(0, 4), (1, 10), (2, 8), (3, 9)]
[(0, 8), (1, 10), (2, 1), (3, 10)]
[(0, 17), (1, 16), (2, 7), (3, 16)]
[(0, 2)]
[(0, 2), (1, 2), (2, 3), (3, 6)]
[(0, 413), (1, 375), (2, 165), (3, 269)]
[(0, 1), (1, 1), (3, 4)]
[(0, 1), (1, 1), (2, 2), (3, 3)]
[(0, 8)]
[(0, 6), (1, 21), (2, 14), (3, 8)]
[(0, 219), (1, 160), (2

/home/foodmap/anaconda2/lib/python2.7/site-packages/scipy/stats/stats.py:2240: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [ ]:
result = query_by_country("united states")  # no results !!! very strange, we can see there are records in match_all
# result = query_by_country("maldives")  # no results
first10 = [x for x in result['hits']['hits'][:10]]
print first10

In [ ]:
get_total_freq_country("italy")

In [130]:
from datetime import datetime, timedelta
from collections import Counter
def datetimetotimestamp(dt):
    return (dt - datetime(1970, 1, 1)).total_seconds()

def split_intervat_in_buckets(interval, daily_frequencies):    
    last_ts = datetimetotimestamp(datetime.strptime(str(max(tpl[0] for tpl in daily_frequencies)), '%Y%m%d'))
    
    tsinterval = interval * 3600 * 24
    interval_dict = Counter()
    for x, freq in daily_frequencies:
        ts = datetimetotimestamp(datetime.strptime(str(x), '%Y%m%d'))
        
        tsid = int((ts - last_ts) / tsinterval)
        interval_dict[tsid] += freq
    
    minkey = min(interval_dict.keys())
    return sorted([(tsid-minkey, freq) for tsid, freq in interval_dict.iteritems()])

In [134]:
query_result = query_by_country("united states")

category_dict = defaultdict(list)
for x in query_result['hits']['hits']:
    category_dict[x["_source"]['category']].append(tuple((x["_source"]['date'], x["_source"]['count'])))
    
for category, values in category_dict.iteritems():
    sorted_counts = sorted(values, key=lambda x: x[0])
    print sorted_counts
    new_intervals = split_intervat_in_buckets(7, sorted_counts)
    print new_intervals
    print "Freq before intervals", category, sum([y for x, y in sorted_counts])
    print "Freq after intervals", category, sum([y for x, y in new_intervals])
    print "Last day popularity", category, category, stats.zscore([y for x, y in sorted_counts])[-1]
    break

[(20170310, 3), (20170311, 21), (20170312, 7), (20170313, 10), (20170314, 17), (20170315, 18), (20170316, 21), (20170317, 11), (20170318, 18), (20170319, 8), (20170320, 27), (20170321, 15), (20170322, 13), (20170323, 10), (20170324, 8), (20170325, 9), (20170326, 13), (20170327, 13), (20170328, 10), (20170329, 17), (20170330, 20), (20170331, 12), (20170401, 10), (20170402, 14), (20170403, 9), (20170404, 23), (20170405, 18), (20170406, 11), (20170407, 11), (20170408, 6), (20170409, 12), (20170410, 16), (20170411, 7), (20170412, 8), (20170413, 12), (20170414, 10), (20170415, 26), (20170416, 8), (20170417, 40), (20170418, 17), (20170419, 17), (20170420, 20), (20170421, 11), (20170422, 15), (20170423, 10), (20170424, 10), (20170425, 13), (20170426, 6), (20170427, 19), (20170428, 21), (20170429, 13), (20170430, 23), (20170501, 8), (20170502, 62), (20170503, 21), (20170504, 17), (20170505, 85), (20170506, 17), (20170507, 19), (20170508, 12), (20170509, 8), (20170510, 17), (20170511, 10), (201

## TRENDS
Idea: plot trend line on scatter/line plots with daily values

https://docs.scipy.org/doc/numpy-1.3.x/reference/generated/numpy.polyfit.html

In [ ]:
y = np.array([ 0.7972,  0.0767,  0.4383,  0.7866,  0.8091, 0.1954,  0.6307,  0.6599,  0.1065,  0.0508]) 
x = np.arange(len(y))

regression = np.polyfit(x, y, 1)
regression
# y = regression[0] * x + regression[1]

import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(y)
new_y = [regression[0]*i+regression[1] for i in x]
plt.plot(new_y)

## ZSCORE
Calculates the z score of each value in the sample, relative to the sample mean and standard deviation.

https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.zscore.html

In [ ]:
import numpy as np
from scipy import stats

a = np.array([ 0.7972,  0.0767,  0.4383,  0.7866,  0.8091, 0.1954,  0.6307,  0.6599,  0.1065,  0.0508])
stats.zscore(a)

### for vini

In [ ]:
import os
import sys
from collections import defaultdict
os.chdir("/home/foodmap/food101/")
from processing.location.locations import Countries

def getCountriesAsList():
    countriesIndex, countriesInfo = Countries.loadFromFile()
    country_list = list()
    for k,v in countriesInfo.iteritems():
        country_list.append(v[0].encode('ascii','ignore'))
    country_list = sorted(country_list)
    return country_list

In [ ]:
a = getCountriesAsList()
print len(a), a

In [ ]:
lst = [ x.strip().replace(' ', '_').lower() for x in categories]
list(set(lst))